In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
# import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

from keras.preprocessing import image

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2


#Tune hyperparameters
num_filter = 36
l = 12
lr = 0.001


fileid='0_001lr36F0_2DRAndAugmentation'
initial_epoch = 0
previously_saved_weights = ''

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [7]:
img_height, img_width, channel

(32, 32, 3)

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr
  
  
def schedule_lr(epoch):
  
  if epoch in [20,21,22,23]:
    lrate = 0.001/2
  elif epoch in [24,25,26,27]:  
    lrate = 0.001/4
  elif epoch in [28,29,30,31]:  
    lrate = 0.001/6
  elif epoch in [32, 33,34,35]:  
    lrate = 0.001/8
  elif epoch in [36,37,38,39]:  
    lrate = 0.001/10
  elif epoch in [40,41]:  
    lrate = 0.001/12
  elif epoch in [42,43]:  
    lrate = 0.001/14
  elif epoch in [44,45]:  
    lrate = 0.001/16
  elif epoch in [46,47]:  
    lrate = 0.001/18
  elif epoch in [48,49]:  
    lrate = 0.001/20
  else:
    lrate = 0.001

    
  return lrate

In [0]:
# determine Loss function and Optimizer
optimizer = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

lr_metric = get_lr_metric(optimizer)


input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [12]:
model = Model(inputs=[input], outputs=[output])

if initial_epoch > 0:
#   load previously save weights
  print('Loading previously saved weights - ', previously_saved_weights)
  model.load_weights(previously_saved_weights)
  
  
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 36)   972         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 36)   144         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 36)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer = optimizer,
              metrics=['accuracy', lr_metric])

In [15]:
# !rm *.*
!ls


datalab


In [0]:
import keras
from keras.callbacks import ModelCheckpoint


filepath=fileid + "-weights-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
logfilename = fileid + "-log.csv"
csv_logger = keras.callbacks.CSVLogger(logfilename, separator=',', append=True)

lr_scheduler = keras.callbacks.LearningRateScheduler(schedule_lr)

callbacks_list = [keras.callbacks.EarlyStopping (monitor='acc', patience=5,),
                 checkpoint, csv_logger, lr_scheduler]

In [0]:
initial_epoch = 0

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    epochs=epochs,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test),
                    workers=4)


# history = model.fit(x_train, y_train,
#                     batch_size=batch_size,
#                     epochs=epochs,
#                     verbose=1,
#                     callbacks=callbacks_list,
#                     validation_data=(x_test, y_test))


Epoch 1/50
391/391 [==============================] - 338s 864ms/step - loss: 1.5570 - acc: 0.4248 - lr: 0.0010 - val_loss: 1.6407 - val_acc: 0.5021 - val_lr: 0.0010

Epoch 00001: val_acc improved from -inf to 0.50210, saving model to 0_001lr36F0_2DRAndAugmentation-weights-01.hdf5
Epoch 2/50
151/391 [==========>...................] - ETA: 3:02 - loss: 1.2096 - acc: 0.5585 - lr: 0.0010391/391 [==============================] - 319s 815ms/step - loss: 1.1498 - acc: 0.5832 - lr: 0.0010 - val_loss: 3.7714 - val_acc: 0.3223 - val_lr: 0.0010

Epoch 00002: val_acc did not improve from 0.50210
Epoch 3/50
248/391 [==================>...........] - ETA: 1:48 - loss: 0.9767 - acc: 0.6505 - lr: 0.0010390/391 [============================>.] - ETA: 0s - loss: 0.9600 - acc: 0.6560 - lr: 0.0010
Epoch 00003: val_acc improved from 0.50210 to 0.63020, saving model to 0_001lr36F0_2DRAndAugmentation-weights-03.hdf5
Epoch 4/50
272/391 [===================>..........] - ETA: 1:30 - loss: 0.8430 - acc: 0.702

223/391 [================>.............] - ETA: 2:07 - loss: 0.2865 - acc: 0.9003 - lr: 2.5000e-04390/391 [============================>.] - ETA: 0s - loss: 0.2842 - acc: 0.9014 - lr: 2.5000e-04
Epoch 00026: val_acc did not improve from 0.87600
Epoch 27/50
324/391 [=======================>......] - ETA: 50s - loss: 0.2832 - acc: 0.9013 - lr: 2.5000e-04391/391 [==============================] - 318s 813ms/step - loss: 0.2847 - acc: 0.9003 - lr: 2.5000e-04 - val_loss: 0.4885 - val_acc: 0.8651 - val_lr: 2.5000e-04

Epoch 00027: val_acc did not improve from 0.87600
Epoch 28/50
288/391 [=====================>........] - ETA: 1:18 - loss: 0.2767 - acc: 0.9033 - lr: 2.5000e-04391/391 [==============================] - 317s 812ms/step - loss: 0.2766 - acc: 0.9039 - lr: 2.5000e-04 - val_loss: 0.5299 - val_acc: 0.8562 - val_lr: 2.5000e-04

Epoch 00028: val_acc did not improve from 0.87600
Epoch 29/50
277/391 [====================>.........] - ETA: 1:26 - loss: 0.2720 - acc: 0.9045 - lr: 1.6667e-

In [15]:
previously_saved_weights = '0_001lr36F0_2DRAndAugmentation-weights-33.hdf5'
initial_epoch = 38
epochs = 40

model = Model(inputs=[input], outputs=[output])

if initial_epoch > 0:
#   load previously save weights
  print('Loading previously saved weights - ', previously_saved_weights)
  model.load_weights(previously_saved_weights)
  
  
model.summary()


Loading previously saved weights -  0_001lr36F0_2DRAndAugmentation-weights-33.hdf5
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 36)   972         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 36)   144         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 36)   0           batch_normalization_1[0][0]      
__________________________

Total params: 995,230
Trainable params: 981,658
Non-trainable params: 13,572
__________________________________________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer = optimizer,
              metrics=['accuracy', lr_metric])

In [0]:
import keras
from keras.callbacks import ModelCheckpoint


filepath=fileid + "-weights-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
logfilename = fileid + "-log.csv"
csv_logger = keras.callbacks.CSVLogger(logfilename, separator=',', append=True)

lr_scheduler = keras.callbacks.LearningRateScheduler(schedule_lr)

callbacks_list = [keras.callbacks.EarlyStopping (monitor='acc', patience=5,),
                 checkpoint, csv_logger, lr_scheduler]

In [20]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
history = model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    epochs=epochs,
                    initial_epoch= initial_epoch,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test),
                    workers=4)

Epoch 39/40
391/391 [==============================] - 335s 858ms/step - loss: 0.2440 - acc: 0.9141 - lr: 1.0000e-04 - val_loss: 0.4335 - val_acc: 0.8825 - val_lr: 1.0000e-04

Epoch 00039: val_acc improved from -inf to 0.88250, saving model to 0_001lr36F0_2DRAndAugmentation-weights-39.hdf5
Epoch 40/40
140/391 [=========>....................] - ETA: 3:10 - loss: 0.2389 - acc: 0.9159 - lr: 1.0000e-04391/391 [==============================] - 317s 811ms/step - loss: 0.2411 - acc: 0.9158 - lr: 1.0000e-04 - val_loss: 0.4699 - val_acc: 0.8763 - val_lr: 1.0000e-04

Epoch 00040: val_acc did not improve from 0.88250


In [21]:
initial_epoch = 40
epochs = 45

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
history = model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    epochs=epochs,
                    initial_epoch= initial_epoch,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test),
                    workers=4)

Epoch 41/45
391/391 [==============================] - 318s 814ms/step - loss: 0.2369 - acc: 0.9168 - lr: 8.3333e-05 - val_loss: 0.4057 - val_acc: 0.8869 - val_lr: 8.3333e-05

Epoch 00041: val_acc improved from 0.88250 to 0.88690, saving model to 0_001lr36F0_2DRAndAugmentation-weights-41.hdf5
Epoch 42/45
140/391 [=========>....................] - ETA: 3:10 - loss: 0.2373 - acc: 0.9160 - lr: 8.3333e-05391/391 [==============================] - 318s 813ms/step - loss: 0.2371 - acc: 0.9175 - lr: 8.3333e-05 - val_loss: 0.4406 - val_acc: 0.8808 - val_lr: 8.3333e-05

Epoch 00042: val_acc did not improve from 0.88690
Epoch 43/45
231/391 [================>.............] - ETA: 2:01 - loss: 0.2324 - acc: 0.9181 - lr: 7.1429e-05391/391 [==============================] - 318s 813ms/step - loss: 0.2359 - acc: 0.9168 - lr: 7.1429e-05 - val_loss: 0.4301 - val_acc: 0.8837 - val_lr: 7.1429e-05

Epoch 00043: val_acc did not improve from 0.88690
Epoch 44/45
260/391 [==================>...........] - ETA

In [12]:
previously_saved_weights = '0_001lr36F0_2DRAndAugmentation-weights-45.hdf5'
initial_epoch = 45
epochs = 50

model = Model(inputs=[input], outputs=[output])

if initial_epoch > 0:
#   load previously save weights
  print('Loading previously saved weights - ', previously_saved_weights)
  model.load_weights(previously_saved_weights)
  
  
model.summary()

Loading previously saved weights -  0_001lr36F0_2DRAndAugmentation-weights-45.hdf5
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 36)   972         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 36)   144         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 36)   0           batch_normalization_1[0][0]      
__________________________

Total params: 995,230
Trainable params: 981,658
Non-trainable params: 13,572
__________________________________________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer = optimizer,
              metrics=['accuracy', lr_metric])

In [0]:
import keras
from keras.callbacks import ModelCheckpoint


filepath=fileid + "-weights-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
logfilename = fileid + "-log.csv"
csv_logger = keras.callbacks.CSVLogger(logfilename, separator=',', append=True)

lr_scheduler = keras.callbacks.LearningRateScheduler(schedule_lr)

callbacks_list = [keras.callbacks.EarlyStopping (monitor='acc', patience=5,),
                 checkpoint, csv_logger, lr_scheduler]

In [15]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
history = model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    epochs=epochs,
                    initial_epoch= initial_epoch,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test),
                    workers=4)

Epoch 46/50
391/391 [==============================] - 336s 861ms/step - loss: 0.2295 - acc: 0.9195 - lr: 6.2500e-05 - val_loss: 0.4231 - val_acc: 0.8851 - val_lr: 6.2500e-05

Epoch 00046: val_acc improved from -inf to 0.88510, saving model to 0_001lr36F0_2DRAndAugmentation-weights-46.hdf5
Epoch 47/50
140/391 [=========>....................] - ETA: 3:10 - loss: 0.2328 - acc: 0.9194 - lr: 5.5556e-05391/391 [==============================] - 318s 812ms/step - loss: 0.2321 - acc: 0.9195 - lr: 5.5556e-05 - val_loss: 0.4125 - val_acc: 0.8877 - val_lr: 5.5556e-05

Epoch 00047: val_acc improved from 0.88510 to 0.88770, saving model to 0_001lr36F0_2DRAndAugmentation-weights-47.hdf5
Epoch 48/50
190/391 [=============>................] - ETA: 2:32 - loss: 0.2253 - acc: 0.9216 - lr: 5.5556e-05391/391 [==============================] - 318s 813ms/step - loss: 0.2266 - acc: 0.9211 - lr: 5.5556e-05 - val_loss: 0.4243 - val_acc: 0.8851 - val_lr: 5.5556e-05

Epoch 00048: val_acc did not improve from 0

In [16]:
!ls
# !rm Adam*.*

0_001lr36F0_2DRAndAugmentation-log.csv
0_001lr36F0_2DRAndAugmentation-weights-01.hdf5
0_001lr36F0_2DRAndAugmentation-weights-03.hdf5
0_001lr36F0_2DRAndAugmentation-weights-05.hdf5
0_001lr36F0_2DRAndAugmentation-weights-07.hdf5
0_001lr36F0_2DRAndAugmentation-weights-08.hdf5
0_001lr36F0_2DRAndAugmentation-weights-09.hdf5
0_001lr36F0_2DRAndAugmentation-weights-10.hdf5
0_001lr36F0_2DRAndAugmentation-weights-16.hdf5
0_001lr36F0_2DRAndAugmentation-weights-20.hdf5
0_001lr36F0_2DRAndAugmentation-weights-21.hdf5
0_001lr36F0_2DRAndAugmentation-weights-25.hdf5
0_001lr36F0_2DRAndAugmentation-weights-29.hdf5
0_001lr36F0_2DRAndAugmentation-weights-33.hdf5
0_001lr36F0_2DRAndAugmentation-weights-39.hdf5
0_001lr36F0_2DRAndAugmentation-weights-41.hdf5
0_001lr36F0_2DRAndAugmentation-weights-45.hdf5
0_001lr36F0_2DRAndAugmentation-weights-46.hdf5
0_001lr36F0_2DRAndAugmentation-weights-47.hdf5
0_001lr36F0_2DRAndAugmentation-weights-50.hdf5
datalab


In [17]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 28s 3ms/step
Test loss: 0.4101718578398228
Test accuracy: 0.8884


In [0]:
from google.colab import files

weightfiles = [
    '0_001lr36F0_2DRAndAugmentation-log.csv',
    '0_001lr36F0_2DRAndAugmentation-weights-01.hdf5',
    '0_001lr36F0_2DRAndAugmentation-weights-03.hdf5',
    '0_001lr36F0_2DRAndAugmentation-weights-05.hdf5',
    '0_001lr36F0_2DRAndAugmentation-weights-07.hdf5',
    '0_001lr36F0_2DRAndAugmentation-weights-08.hdf5',
    '0_001lr36F0_2DRAndAugmentation-weights-09.hdf5',
    '0_001lr36F0_2DRAndAugmentation-weights-10.hdf5',
    '0_001lr36F0_2DRAndAugmentation-weights-16.hdf5',
    '0_001lr36F0_2DRAndAugmentation-weights-20.hdf5',
    '0_001lr36F0_2DRAndAugmentation-weights-21.hdf5',
    '0_001lr36F0_2DRAndAugmentation-weights-25.hdf5',
    '0_001lr36F0_2DRAndAugmentation-weights-29.hdf5',
    '0_001lr36F0_2DRAndAugmentation-weights-33.hdf5',
    '0_001lr36F0_2DRAndAugmentation-weights-39.hdf5',
    '0_001lr36F0_2DRAndAugmentation-weights-41.hdf5',
    '0_001lr36F0_2DRAndAugmentation-weights-45.hdf5',
    '0_001lr36F0_2DRAndAugmentation-weights-46.hdf5',
    '0_001lr36F0_2DRAndAugmentation-weights-47.hdf5',
    '0_001lr36F0_2DRAndAugmentation-weights-50.hdf5']

for filepath in weightfiles:
  files.download(filepath)